# Dataset Formatting

In [9]:
from collections import defaultdict
from itertools import chain
import json
import ijson 
import argparse
from collections import Counter
import random
from argparse import Namespace
from tqdm.notebook import tqdm
import torch
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer
import sys
sys.path.insert(0,'..')
from utils import *

In [10]:
# msmarco, eli5, wow
msmarco_path = '/ssd003/projects/aieng/conversational_ai/data/MSMARCO'
eli5_path = "/ssd003/projects/aieng/conversational_ai/data/Eli5"
wow_path = "/ssd003/projects/aieng/conversational_ai/data/Wizard_of_Wikipedia/"

### MSMarco

In [6]:
data = json.load(open(msmarco_path + "/marco_train_data.json"))
jtopy=json.dumps(data) #json.dumps take a dictionary as input and returns a string as output.
msmarco_data=json.loads(jtopy)

In [67]:
msmarco_idx = 0

In [83]:
# check if shirley concatenates passage, query during tokenization
msmarco_data[1000]

{'context': [{'is_selected': 0,
   'passage_text': "Beast Sports Nutrition's Aminolytes is designed with the most advanced amino matrix that contains electrolytes and aminos to provide muscle support for longer endurance and faster recovery post-workout.",
   'url': 'http://www.esupplements.com/best-intra-workout-supplements/'},
  {'is_selected': 0,
   'passage_text': "#7 - Purus Labs NOXygen. Formulated to be a intra-workout accelerator, Purus Labs' NOXygen amplifies the flow of blood and oxygen without stimulant properties, creatine, or dye making it a perfect supplement to add to your daily routine.",
   'url': 'http://www.esupplements.com/best-intra-workout-supplements/'},
  {'is_selected': 0,
   'passage_text': 'Intra means within, as in a transfer within the bank. Your bank may have an agreement where you designate your accounts to and from which funds may go. A checking and saving … s account at the same bank are examples. 12 people found this useful.',
   'url': 'http://www.ans

In [77]:
msmarco_sample = msmarco_data[msmarco_idx].copy()
msmarco_sample["context"] = msmarco_sample["context"][:2]

In [68]:
msmarco_data[msmarco_idx].keys()

dict_keys(['context', 'query_type', 'utterances'])

In [78]:
msmarco_sample

{'context': [{'is_selected': 1,
   'passage_text': 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.',
   'url': 'http://www.pitt.edu/~sdb14/atombomb.html'},
  {'is_selected': 0,
   'passage_text': 'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.',
   'url': 'http://www.osti.gov/accomplishments/manhattan_story.html'}],
 'query_type': 'DESCRIPTION',
 'utterances': [{'history': [')what was the immediate impact of the success of the manhattan project?'],
   'candidates': ["I'm sorry, I don't know.",
    'The average car insurance rate in Florida is $1,823 a year.',
    'The immediate impact 

### ELI5

In [90]:
msmarco_data[0].keys()

dict_keys(['query_id', 'context', 'query_type', 'category', 'utterances'])

In [87]:
data = json.load(open(eli5_path + "/Eli5_model_files/eli5_train.json"))
jtopy=json.dumps(data) #json.dumps take a dictionary as input and returns a string as output.
msmarco_data=json.loads(jtopy)

In [89]:
msmarco_data[0]

{'query_id': 'cpu5fz',
 'context': [{'wikipedia_id': '40579087',
   'title': 'Quotient stack',
   'section': '',
   'start_paragraph_id': 1,
   'start_character': 0,
   'end_paragraph_id': 1,
   'end_character': 246,
   'bleu_score': None,
   'meta': None,
   'passage_text': 'In algebraic geometry, a quotient stack is a stack that parametrizes equivariant objects. Geometrically, it generalizes a quotient of a scheme or a variety by a group: a quotient variety, say, would be a coarse approximation of a quotient stack.\n',
   'is_selected': 1},
  {'wikipedia_id': '27548536',
   'title': 'The Quants',
   'section': 'Section::::Synopsis.\n',
   'start_paragraph_id': 5,
   'start_character': 0,
   'end_paragraph_id': 5,
   'end_character': 586,
   'bleu_score': None,
   'meta': None,
   'passage_text': 'The introduction to "The Quants" describes the real-life, annual, high-stakes poker match between Wall Street\'s hedge fund managers, comparing their trading styles to their poker strategies

In [88]:
# with open(eli5_path + "/Eli5_model_files/eli5_train.json") as f:
#     train_data = json.load(f)
#     eli5_train = json.loads(train_data)
    
# # with open(eli5_path + "/raw/eli5_categories.json") as f:
# #     cat_data = json.load(f)
# #     eli5_cat = json.loads(cat_data)

In [53]:
eli5_idx = 0

In [54]:
eli5_train[eli5_idx].keys()

dict_keys(['id', 'input', 'output', 'answer'])

In [55]:
eli5_cat[eli5_idx].keys()

dict_keys(['id', 'input', 'answers_list', 'passages', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'])

In [64]:
eli5_sample = eli5_train[eli5_idx].copy()
eli5_sample["output"] = eli5_sample["output"][:2]

In [65]:
eli5_sample

{'id': '32wvn8',
 'input': "what's the difference between a forest and a wood?",
 'output': [{'text': 'Wood is divided, according to its botanical origin, into two kinds: softwoods, from coniferous trees, and hardwoods, from broad-leaved trees. Softwoods are lighter and generally simple in structure, whereas hardwoods are harder and more complex. However, in Australia, "softwood" generally describes rain forest trees, and "hardwood" describes Sclerophyll species ("Eucalyptus" "spp").\n',
   'title': 'Wood drying',
   'wikipedia_id': '4396843'},
  {'text': 'Woodland is defined by Chambers English dictionary as "land covered with wood" i.e. dominated by tree species. Forestry is defined as "1. the science and art of planting, tending and managing forests; 2. Forest country". This implies that forests have been planted by mankind for a variety of purposes, but mostly for exploitation for timber and pulp for the paper industry. The majority of Forests in Wales were planted by the British F

### WoW

In [15]:
with open(wow_path + "/wizard_of_wikipedia.json") as f:
    wow_data = json.load(f)
    wow_data = json.loads(wow_data)

In [26]:
wow_data

{'id': {'0': '6bc20426-99d6-11ea-8a20-773209e30a7b_0',
  '1': '5592954e-99d6-11ea-8a20-773209e30a7b_0',
  '2': '536ab85a-99d6-11ea-8a20-773209e30a7b_2',
  '3': '81a9e09c-99d6-11ea-8a20-773209e30a7b_3',
  '4': '5305dbba-99d6-11ea-8a20-773209e30a7b_3',
  '5': '70777d5c-99d6-11ea-8a20-773209e30a7b_3',
  '6': '58f207e2-99d6-11ea-8a20-773209e30a7b_4',
  '7': '67e1456a-99d6-11ea-8a20-773209e30a7b_2',
  '8': '612cb682-99d6-11ea-8a20-773209e30a7b_1',
  '9': '77250ee4-99d6-11ea-8a20-773209e30a7b_4',
  '10': '844dd880-99d6-11ea-8a20-773209e30a7b_0',
  '11': '66d98542-99d6-11ea-8a20-773209e30a7b_2',
  '12': '6cdc904c-99d6-11ea-8a20-773209e30a7b_3',
  '13': '52e9b336-99d6-11ea-8a20-773209e30a7b_3',
  '14': '5b52cb98-99d6-11ea-8a20-773209e30a7b_1',
  '15': '8572d0a8-99d6-11ea-8a20-773209e30a7b_3',
  '16': '699dda1c-99d6-11ea-8a20-773209e30a7b_3',
  '17': '8a806376-99d6-11ea-8a20-773209e30a7b_2',
  '18': '655ac8ca-99d6-11ea-8a20-773209e30a7b_0',
  '19': '8379ed5e-99d6-11ea-8a20-773209e30a7b_2',
  '2

In [21]:
wow_data.keys()

dict_keys(['id', 'input', 'meta', 'output', 'answer', 'passages_text'])

In [27]:
len(wow_data['id'])

57384

In [28]:
wow_idx = '1'

In [29]:
wow_data['id'][wow_idx]

'5592954e-99d6-11ea-8a20-773209e30a7b_0'

In [30]:
wow_data['input'][wow_idx]

'I would love to be a surgeon when I grow up.'

In [31]:
wow_data['meta'][wow_idx]

{'left_context': '',
 'mention': '',
 'right_context': '',
 'partial_evidence': [],
 'obj_surface': [],
 'sub_surface': [],
 'subj_aliases': [],
 'template_questions': []}

In [32]:
wow_data['output'][wow_idx]

[{'answer': 'Me too. Performing surgical operations on people sounds fun!',
  'meta': {'score': -1},
  'provenance': [{'bleu_score': 0.9121679068,
    'start_character': 3,
    'start_paragraph_id': 1,
    'end_character': 78,
    'end_paragraph_id': 1,
    'meta': {'fever_page_id': '',
     'fever_sentence_id': -1,
     'annotation_id': '-1',
     'yes_no_answer': '',
     'evidence_span': []},
    'section': 'Section::::Abstract.',
    'title': 'Surgeon',
    'wikipedia_id': '9104734'}]}]

In [33]:
wow_data['answer'][wow_idx]

'Me too. Performing surgical operations on people sounds fun!'

In [34]:
wow_data['passages_text'][wow_idx]

'In modern medicine, a surgeon is a physician who performs surgical operations. There are also surgeons in podiatry, dentistry maxillofacial surgeon and the veterinary fields.\n'